#### Exercise 1

a) Read set_1.xlsx, set_2.xlsx and set_3.xlsx files.

b) Calculate estimator of correlation matrix for each data set.

c) Do they need to be adjusted? Please justify your choice.

In [11]:
# Exercise 1 – reading the data sets
import os
import pandas as pd
import numpy as np

# List of input files
files = ['set_1.xlsx', 'set_2.xlsx', 'set_3.xlsx']

# Read all data sets into a list
data_list = []
for f in files:
    path = os.path.join(os.getcwd(), f)
    df = pd.read_excel(path)
    data_list.append(df)

# Optional: check shapes to confirm successful reading
for i, df in enumerate(data_list, start=1):
    print("\033[1m" + f'set_{i} shape:' + "\033[0m", df.shape)


set_1 shape: (756, 5)
set_2 shape: (756, 5)
set_3 shape: (756, 5)


In [12]:
# Exercise 1 – correlation matrices

corr_list = []

for i, df in enumerate(data_list, start=1):
    # Drop non-numeric date column (named 'Data' in the files)
    df_num = df.drop(columns=['Data'])
    
    # Sample correlation matrix
    corr_i = df_num.corr()
    corr_list.append(corr_i)
    
    print('')
    print("\033[1m" + f'Correlation matrix for set_{i}:' + "\033[0m")
    print(corr_i)



Correlation matrix for set_1:
          NDAQ       DJI       FTM       SPX
NDAQ  1.000000  0.433603  0.699070  0.765190
DJI   0.433603  1.000000  0.918722  0.981461
FTM   0.699070  0.918722  1.000000  0.844147
SPX   0.765190  0.981461  0.844147  1.000000

Correlation matrix for set_2:
          NDAQ       DJI       FTM       SPX
NDAQ  1.000000  0.918569  0.692465  0.971234
DJI   0.918569  1.000000  0.885357  0.978796
FTM   0.692465  0.885357  1.000000  0.791672
SPX   0.971234  0.978796  0.791672  1.000000

Correlation matrix for set_3:
      NDAQ  DJI  FTM  SPX
NDAQ   1.0 -1.0  1.0  1.0
DJI   -1.0  1.0 -1.0 -1.0
FTM    1.0 -1.0  1.0  1.0
SPX    1.0 -1.0  1.0  1.0


In [13]:
# Exercise 1 – check if each estimator needs adjustment
from scipy.linalg import eigh
from numpy.linalg import norm

# Threshold for "numerically close to singular"
r_limit = 10**(-8)

for i, corr_i in enumerate(corr_list, start=1):
    # Eigenvalues of the correlation estimator
    eigenvals_corr_i, _ = eigh(corr_i)
    
    print('')
    print("\033[1m" + f'set_{i} – eigenvalues of corr estimator:' + "\033[0m")
    print(np.round(eigenvals_corr_i, 10))
    
    min_ev = np.min(eigenvals_corr_i)
    
    if min_ev <= 0:
        print('-> Matrix is not positive definite (min eigenvalue <= 0). Needs adjustment.')
    elif min_ev < r_limit:
        print(f'-> Matrix is numerically close to singular (min eigenvalue < {r_limit}).')
        print('   In practice it is safer to adjust.')
    else:
        print('-> Matrix is positive definite. No adjustment strictly required.')



set_1 – eigenvalues of corr estimator:
[-0.09661063  0.16329641  0.58956346  3.34375076]
-> Matrix is not positive definite (min eigenvalue <= 0). Needs adjustment.

set_2 – eigenvalues of corr estimator:
[3.09153030e-03 2.83271775e-02 3.41913647e-01 3.62666764e+00]
-> Matrix is positive definite. No adjustment strictly required.

set_3 – eigenvalues of corr estimator:
[-0. -0.  0.  4.]
-> Matrix is not positive definite (min eigenvalue <= 0). Needs adjustment.


Conclusion for Exercise 1c

set 1: has a negative eigenvalue → the correlation matrix is not positive definite → it must be adjusted.

set 2: all eigenvalues are positive and not close to zero → the matrix is positive definite → no strict adjustment is required, although one can still apply a cleaning method.

set 3: eigenvalues are (up to rounding) non-positive except one → the matrix is not positive definite → it must be adjusted.

#### Exercise 2

Write a following functions:

##### a) Spectral Decomposition Method
    Input arguments: 
        - Estimator of correlation matrix
        - Small parameter 𝜀>0 (default value = 10^-8)
    
    Output:
        - Adjusted estimator correlation matrix with Spectrtal Decomposition Method
        

##### b) Alternating Projection Method
    Input arguments: 
        - Estimator of correlation matrix
        - Tolerance value (default value = 10^-12)
        - 𝜏 parameter (default value = 10^-8)
    
    Output:
        - Adjusted estimator correlation matrix with Gradient Method
        - Number of iterations needed to find adjusted correlation matrix
        - Distance between original and adjusted matrices

In [14]:
# Exercise 2a – Spectral Decomposition Method

from scipy.linalg import eigh

def spectral_decomposition(corr_matrix, epsilon=10**(-8)):
    """
    Adjust a correlation matrix using the Spectral Decomposition Method.

    Parameters
    ----------
    corr_matrix : array-like (n x n)
        Sample correlation matrix.
    epsilon : float, optional
        Small positive floor for eigenvalues. Eigenvalues smaller than epsilon
        are replaced by epsilon.

    Returns
    -------
    C_adj : ndarray (n x n)
        Adjusted correlation matrix: symmetric, positive definite,
        with ones on the diagonal.
    """
    # Work on a NumPy array copy
    C = np.array(corr_matrix, dtype=float)
    
    # Eigenvalues and eigenvectors of C
    eigenvals_C, eigenvecs_C = eigh(C)
    
    # Floor small / negative eigenvalues
    eigenvals_C_adj = np.where(eigenvals_C < epsilon, epsilon, eigenvals_C)
    
    # Rebuild adjusted matrix
    C_adj = eigenvecs_C @ np.diag(eigenvals_C_adj) @ eigenvecs_C.T
    
    # Rescale to get ones on the main diagonal (turn covariance into correlation)
    d = np.sqrt(np.diag(C_adj))
    C_adj = C_adj / d[:, None] / d[None, :]
    
    return C_adj


In [15]:
# Apply Spectral Decomposition to each correlation matrix

corr_adj_spectral_list = []

for i, corr_i in enumerate(corr_list, start=1):
    C_adj_i = spectral_decomposition(corr_i)
    corr_adj_spectral_list.append(C_adj_i)
    
    print('')
    print("\033[1m" + f'set_{i} – adjusted corr (Spectral Decomposition):' + "\033[0m")
    print(np.round(C_adj_i, 4))



set_1 – adjusted corr (Spectral Decomposition):
[[1.     0.4442 0.6802 0.7288]
 [0.4442 1.     0.8756 0.9109]
 [0.6802 0.8756 1.     0.8444]
 [0.7288 0.9109 0.8444 1.    ]]

set_2 – adjusted corr (Spectral Decomposition):
[[1.     0.9186 0.6925 0.9712]
 [0.9186 1.     0.8854 0.9788]
 [0.6925 0.8854 1.     0.7917]
 [0.9712 0.9788 0.7917 1.    ]]

set_3 – adjusted corr (Spectral Decomposition):
[[ 1. -1.  1.  1.]
 [-1.  1. -1. -1.]
 [ 1. -1.  1.  1.]
 [ 1. -1.  1.  1.]]


In [16]:
# Exercise 2b – Alternating Projection Method

from scipy.linalg import eigh
from numpy.linalg import norm

def alternating_projection(corr_matrix, tol=1e-12, tau=1e-8, max_iter=5000):
    """
    Adjust a correlation matrix using the Alternating Projection Method
    (Dykstra's algorithm) to find the nearest correlation matrix.

    Parameters
    ----------
    corr_matrix : array-like (n x n)
        Sample correlation matrix.
    tol : float, optional
        Tolerance for convergence (Frobenius norm of change between iterates).
    tau : float, optional
        Eigenvalue floor when projecting onto the PSD cone.
    max_iter : int, optional
        Maximum number of iterations.

    Returns
    -------
    X : ndarray (n x n)
        Adjusted correlation matrix (symmetric, PSD, ones on diagonal).
    k : int
        Number of iterations performed.
    distance : float
        Frobenius norm of the difference between original and adjusted matrices.
    """
    # Work on a NumPy array copy
    A = np.array(corr_matrix, dtype=float)
    n = A.shape[0]
    
    # Step 0 (Initialization)
    Y = A.copy()
    delta_S = np.zeros_like(A)
    
    # Target diagonal (ones)
    b = np.ones(n)
    
    X = Y.copy()
    
    for k in range(1, max_iter + 1):
        X_prev = X.copy()
        
        # Step 1 (Dykstra's correction: project onto PSD cone)
        R = Y - delta_S
        
        # Eigen-decomposition of R
        eigenvals_R, eigenvecs_R = eigh(R)
        
        # Floor eigenvalues below tau to ensure PSD
        eigenvals_R_adj = np.where(eigenvals_R < tau, tau, eigenvals_R)
        X = eigenvecs_R @ np.diag(eigenvals_R_adj) @ eigenvecs_R.T
        
        # Step 2 (Update Dykstra's correction)
        delta_S = X - R
        
        # Step 3 (Project onto the set of matrices with unit diagonal)
        # Keep off-diagonal entries and force diagonal to be exactly one
        np.fill_diagonal(X, 1.0)
        
        # Prepare next iterate
        Y = X
        
        # Check convergence: change between successive iterates
        diff = norm(X - X_prev, ord='fro')
        if diff <= tol:
            print('iterations:', k)
            print('||X_k - X_{k-1}||_F:', diff)
            break
    else:
        # If we exit the for-loop without break, max_iter was reached
        print('WARNING: max_iter reached before convergence')
        print('iterations:', max_iter)
        print('||X_k - X_{k-1}||_F:', diff)
    
    # Symmetrize just in case of small numerical asymmetries
    X = 0.5 * (X + X.T)
    
    # Distance between original and adjusted matrices
    distance = norm(A - X, ord='fro')
    
    return X, k, distance


In [17]:
# Apply Alternating Projection to each correlation matrix

alt_adj_list = []

for i, corr_i in enumerate(corr_list, start=1):
    X_adj_i, n_iter_i, dist_i = alternating_projection(corr_i)
    alt_adj_list.append(X_adj_i)
    
    print('')
    print("\033[1m" + f'set_{i} – adjusted corr (Alternating Projection):' + "\033[0m")
    print(np.round(X_adj_i, 4))
    print('number of iterations:', n_iter_i)
    print('distance to original:', dist_i)


iterations: 27
||X_k - X_{k-1}||_F: 5.085153811909136e-13

set_1 – adjusted corr (Alternating Projection):
[[1.     0.4661 0.6845 0.7357]
 [0.4661 1.     0.89   0.9235]
 [0.6845 0.89   1.     0.8702]
 [0.7357 0.9235 0.8702 1.    ]]
number of iterations: 27
distance to original: 0.11834596498549561
iterations: 1
||X_k - X_{k-1}||_F: 2.0621477437872914e-15

set_2 – adjusted corr (Alternating Projection):
[[1.     0.9186 0.6925 0.9712]
 [0.9186 1.     0.8854 0.9788]
 [0.6925 0.8854 1.     0.7917]
 [0.9712 0.9788 0.7917 1.    ]]
number of iterations: 1
distance to original: 1.967320211922957e-15
iterations: 33
||X_k - X_{k-1}||_F: 8.699483193199243e-13

set_3 – adjusted corr (Alternating Projection):
[[ 1. -1.  1.  1.]
 [-1.  1. -1. -1.]
 [ 1. -1.  1.  1.]
 [ 1. -1.  1.  1.]]
number of iterations: 33
distance to original: 3.463840648048236e-08
